In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json
import pandas as pd
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer

# 1. JSON 파일 로드
json_path = '/content/drive/MyDrive/trinity_data/restaurants_data_prototype.json'
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
df = pd.DataFrame(data)

# 2. 기본 컬럼 타입 변환
# review: 숫자형으로 변환 (변환 실패 시 NaN)
df['review'] = pd.to_numeric(df['review'], errors='coerce')

# category_id: 정수형으로 변환 시도 (실패하면 원본 유지)
# 한글 카테고리명을 원하는 숫자로 매핑하는 딕셔너리
category_mapping = {
    "중식": 0,
    "일식집": 1,
    "브런치카페": 2,
    "파스타": 3,
    "이탈리안": 4,
    "이자카야": 5,
    "한식집": 6,
    "치킨": 7,
    "스테이크": 8,
    "고깃집": 9,
    "다이닝바": 10,
    "오마카세": 11
}

def convert_category(cat):
    # 먼저 매핑 딕셔너리를 통해 변환 시도
    if cat in category_mapping:
        return category_mapping[cat]
    else:
        try:
            return int(cat)
        except:
            return cat

df['category_id'] = df['category_id'].apply(convert_category)


# phone_number: 숫자형(또는 실수)일 경우 정수 변환 후 문자열로 변경
def format_phone(num):
    if pd.isna(num):
        return ""
    try:
        num_int = int(num)
        return str(num_int)
    except:
        return str(num)
df['phone_number'] = df['phone_number'].apply(format_phone)

# 3. 편의시설(convenience) 처리 및 인코딩
# 원본 편의시설 컬럼은 그대로 유지합니다.
convenience_counter = Counter()
def normalize_convenience(val):
    items = val.split('\n')
    normalized_items = []
    for item in items:
        normalized_item = item.strip()
        if normalized_item == "정보 없음":
            normalized_item = "편의시설 정보 없음"
        normalized_items.append(normalized_item)
        convenience_counter[normalized_item] += 1
    return normalized_items

# 결측치는 빈 문자열("")로 처리한 후 리스트 생성
df['convenience_list'] = df['convenience'].fillna("").apply(lambda x: normalize_convenience(x) if x != "" else [])

# 다중 원-핫 인코딩 (MultiLabelBinarizer)
mlb_conv = MultiLabelBinarizer()
conv_encoded = mlb_conv.fit_transform(df['convenience_list'])
conv_encoded_df = pd.DataFrame(conv_encoded,
                               columns=[f"conv_{col}" for col in mlb_conv.classes_],
                               index=df.index)

# 4. 유의사항(caution) 처리 및 인코딩
caution_counter = Counter()
def normalize_caution(val):
    items = val.split(',')
    normalized_items = []
    for item in items:
        normalized_item = item.strip()
        if normalized_item == "정보 없음":
            normalized_item = "유의사항 정보 없음"
        normalized_items.append(normalized_item)
        caution_counter[normalized_item] += 1
    return normalized_items

df['caution_list'] = df['caution'].fillna("").apply(lambda x: normalize_caution(x) if x != "" else [])
mlb_caution = MultiLabelBinarizer()
caution_encoded = mlb_caution.fit_transform(df['caution_list'])
caution_encoded_df = pd.DataFrame(caution_encoded,
                                  columns=[f"caution_{col}" for col in mlb_caution.classes_],
                                  index=df.index)

# 5. expanded_days를 이용한 operating_days_count 계산
def count_operating_days(expanded_days):
    day_order = ["월", "화", "수", "목", "금", "토", "일"]
    if not isinstance(expanded_days, str) or expanded_days.strip() == "":
        return None
    expanded_days = expanded_days.strip()
    if "~" in expanded_days:
        parts = expanded_days.split("~")
        if len(parts) != 2:
            return None
        start = parts[0].strip()
        end = parts[1].strip()
        if start in day_order and end in day_order:
            start_idx = day_order.index(start)
            end_idx = day_order.index(end)
            if start_idx <= end_idx:
                return end_idx - start_idx + 1
            else:
                # wrap-around (예: "토~화")
                return (7 - start_idx) + (end_idx + 1)
        else:
            return None
    else:
        days = [d.strip() for d in expanded_days.split(",") if d.strip() != ""]
        return len(days)
df['operating_days_count'] = df['expanded_days'].apply(lambda x: count_operating_days(x) if pd.notna(x) else None)

# 6. time_range 처리: open_time, close_time 추출
df['open_time'] = df['time_range'].apply(lambda x: x.split(" ~ ")[0] if isinstance(x, str) and " ~ " in x else None)
df['close_time'] = df['time_range'].apply(lambda x: x.split(" ~ ")[1] if isinstance(x, str) and " ~ " in x else None)

# 시간 문자열("HH:MM")을 분으로 변환하는 함수 (특히 "24:00"은 1440분으로 처리)
def convert_to_minutes(time_str):
    if not time_str:
        return None
    if time_str == "24:00":
        return 1440
    try:
        h, m = map(int, time_str.split(":"))
        return h * 60 + m
    except:
        return None

df['open_minutes'] = df['open_time'].apply(convert_to_minutes)
df['close_minutes'] = df['close_time'].apply(convert_to_minutes)

# 영업시간(분) 및 시간 단위 계산
def compute_duration(row):
    if pd.isna(row['open_minutes']) or pd.isna(row['close_minutes']):
        return None
    open_m = row['open_minutes']
    close_m = row['close_minutes']
    if close_m >= open_m:
        return close_m - open_m
    else:
        return (24 * 60 - open_m) + close_m

df['duration'] = df.apply(compute_duration, axis=1)
df['duration_hours'] = df['duration'] / 60.0

# 7. open_hour, close_hour 계산
# open_hour: open_time의 시(hour) 부분, close_hour: close_time의 시(hour) (단, "24:00"이면 null 처리)
df['open_hour'] = df['open_time'].apply(lambda x: float(x.split(":")[0]) if x and ":" in x else None)
df['close_hour'] = df['close_time'].apply(lambda x: None if x == "24:00" else (float(x.split(":")[0]) if x and ":" in x else None))

# 8. 인코딩된 편의시설, 유의사항 컬럼 병합
df = pd.concat([df, conv_encoded_df, caution_encoded_df], axis=1)

# 인코딩 후 불필요한 리스트 컬럼 제거
df.drop(columns=['convenience_list', 'caution_list'], inplace=True)

# 9. 최종 출력할 컬럼만 선택
final_columns = [
    "id", "name", "category_id", "score", "review", "address",
    "operating_hour", "expanded_days", "open_time", "close_time",
    "duration", "duration_hours", "time_range", "phone_number",
    "image_urls", "convenience", "caution", "is_deleted",
    "operating_days_count", "open_hour", "close_hour"
]

# 인코딩된 편의시설 컬럼 (접두어 conv_)와 유의사항 컬럼 (접두어 caution_) 추가
# 단, "conv_편의시설 정보 없음"과 "caution_유의사항 정보 없음"은 제외합니다.
conv_cols = [col for col in df.columns if col.startswith("conv_") and col != "conv_편의시설 정보 없음"]
caution_cols = [col for col in df.columns if col.startswith("caution_") and col != "caution_유의사항 정보 없음"]

final_columns += conv_cols + caution_cols

df_final = df[final_columns]

# 최종 결과 확인 (원하는 순서의 컬럼만 존재)
print(df_final.head(10))


      id           name  category_id  score  review  \
0  00-01    이가네양꼬치 판교본점            0    4.0     570   
1  00-02      동청담 삼평직영점            0    3.2      39   
2  00-03         차알 판교점            0    2.9     690   
3  00-04        몽중헌 판교점            0    3.7     356   
4  00-05          최고집짬뽕            0    3.3      65   
5  00-06          청계산수타            0    3.3      16   
6  00-07  신승반점 현대백화점판교점            0    3.5     379   
7  00-08         베이징스토리            0    2.6      51   
8  00-09       마라공방 판교점            0    3.7     121   
9  00-10      명품조박사짬뽕짜장            0    3.7      36   

                                       address     operating_hour  \
0            경기 성남시 분당구 분당내곡로 155 KCC웰츠타워 104호   매일 12:00 ~ 24:00   
1      경기 성남시 분당구 대왕판교로606번길 45 푸르지오시티 2층 206호   매일 15:00 ~ 17:00   
2  경기 성남시 분당구 동판교로177번길 25 판교아비뉴프랑 2층 214-215호   매일 15:00 ~ 17:00   
3              경기 성남시 분당구 분당내곡로 131 판교테크원타워 2층   매일 15:00 ~ 17:30   
4                     경기 성남시 분당구 판교역로10번길 12-9   

In [3]:
!pip install catboost

In [4]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [7]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files  # Colab 환경에서 파일 다운로드를 위해

# Scikit-learn 및 관련 라이브러리 임포트
from sklearn.experimental import enable_iterative_imputer  # 반드시 import 필요
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.linear_model import Ridge, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#####################################
# 사용자 정의 추천 보정 가중치 (조정 가능, 올리면 영향력이 커지고, 내리면 작아짐)
#####################################

# 이 값은 리뷰 수나 리뷰 점수(로그 변환된 값 등)가 최종 평점에 얼마나 큰 영향을 줄지를 결정, 기존 0.1보다 약간 높임
review_weight = 0.4

# 이 값은 유의사항(positive, negative caution 항목들: 예를 들어 "예약가능", "포장가능" 등 긍정적인 항목과 "예약불가", "배달불가" 등 부정적인 항목)의 차이가 최종 평점에 미치는 영향, 기존 0.2보다 약간 낮춤
caution_weight = 0.15

# 편의시설 컬럼들은 one-hot 인코딩되어 각 식당에 대해 해당 편의시설이 있으면 1, 없으면 0
convenience_weight = 0.15  # 편의시설 보정 가중치

#####################################
# 1. 데이터 로드 및 기본 전처리
#####################################
# 전처리된 DataFrame인 df_final을 바로 사용
data = df_final.copy()

# 필수 컬럼: duration_hours, conv_WIFI, conv_주차, caution_예약가능, category_id
required_cols = ['duration_hours', 'conv_WIFI', 'conv_주차', 'caution_예약가능', 'category_id']
data.dropna(subset=required_cols, inplace=True)

# 도메인 규칙:
# 평점을 제공하지 않은 식당은 score가 0으로 기록되어 있다고 가정.
# 만약 score와 review가 모두 0이면, 평점 미제공으로 간주하여 score를 NaN으로 처리.
data.loc[(data['score'] == 0) & (data['review'] == 0), 'score'] = np.nan

# indicator: 평점을 제공했는지 여부
data['score_provided'] = data['score'].notna().astype(int)

#####################################
# 2. Imputation: 결측치 보완 (가상 평점 산출)
#####################################
impute_cols = ['score', 'review']
imputer = IterativeImputer(estimator=BayesianRidge(), random_state=42, max_iter=10, initial_strategy='median')
data[impute_cols] = imputer.fit_transform(data[impute_cols])
# 도메인 최대 평점 5.0으로 클리핑
data.loc[data['score'] > 5, 'score'] = 5.0

#####################################
# 3. 사용자 입력 및 선호 카테고리 필터링
#####################################
user_id = input("사용자 ID를 입력하세요: ")

preferred_categories_input = input("선호하는 식당 카테고리 이름(최대 3개, 콤마로 구분)을 입력하세요: ")
preferred_categories_list = [cat.strip() for cat in preferred_categories_input.split(",")][:3]

category_mapping = {
    "중식": 0, "일식": 1, "브런치": 2, "파스타": 3, "이탈리안": 4,
    "이자카야": 5, "한식": 6, "치킨": 7, "스테이크": 8,
    "고깃집": 9, "다이닝바": 10, "오마카세": 11
}
preferred_category_ids = [category_mapping[cat] for cat in preferred_categories_list if cat in category_mapping]

if not preferred_category_ids:
    print("유효한 카테고리 입력이 없어 프로그램을 종료합니다.")
    exit()

data_filtered = data[data['category_id'].isin(preferred_category_ids)].copy()
if data_filtered.empty:
    print("해당 선호 카테고리에 해당하는 식당 데이터가 없습니다.")
    exit()

#####################################
# 4. 피처 엔지니어링: 비선형 변환 및 상호작용 피처 추가
#####################################
# 기존 피처: ['review', 'duration_hours', 'conv_WIFI', 'conv_주차', 'caution_예약가능']
# 추가 피처: 로그 변환된 review와 review와 duration_hours의 상호작용
data_filtered['log_review'] = np.log(data_filtered['review'] + 1)
data_filtered['review_duration'] = data_filtered['review'] * data_filtered['duration_hours']

# 최종 모델 피처 목록에 기존 피처와 새로 만든 피처 포함
model_features = ['review', 'duration_hours', 'conv_WIFI', 'conv_주차', 'caution_예약가능', 'log_review', 'review_duration']
target = 'score'

X_all = data_filtered[model_features].copy()
y_all = data_filtered[target]

#####################################
# 5. 특성 스케일링 및 데이터 분할
#####################################
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all_scaled, y_all, test_size=0.2, random_state=42)

#####################################
# 6. 모델 학습 및 하이퍼파라미터 튜닝 (cv=3, 축소된 범위)
#####################################
# Model 1: Ridge
param_grid_ridge = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]}
ridge = Ridge()
grid_search_ridge = GridSearchCV(ridge, param_grid_ridge, cv=3, scoring='r2')
grid_search_ridge.fit(X_train_all, y_train_all)
best_ridge = grid_search_ridge.best_estimator_

# Model 2: RandomForest
rf = RandomForestRegressor(random_state=42)
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='r2')
grid_search_rf.fit(X_train_all, y_train_all)
best_rf = grid_search_rf.best_estimator_

# Model 3: XGBoost
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
param_grid_xgb = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1]
}
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, scoring='r2')
grid_search_xgb.fit(X_train_all, y_train_all)
best_xgb = grid_search_xgb.best_estimator_

# Model 4: LightGBM
lgb_model = lgb.LGBMRegressor(random_state=42, verbose=-1, min_split_gain=0)
param_grid_lgb = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5, 7, -1],
    'learning_rate': [0.01, 0.1]
}
grid_search_lgb = GridSearchCV(lgb_model, param_grid_lgb, cv=3, scoring='r2')
grid_search_lgb.fit(X_train_all, y_train_all)
best_lgb = grid_search_lgb.best_estimator_

# Model 5: CatBoost
cat_model = CatBoostRegressor(random_state=42, verbose=0)
param_grid_cat = {
    'iterations': [50, 100],
    'depth': [3, 5],
    'learning_rate': [0.01, 0.1]
}
grid_search_cat = GridSearchCV(cat_model, param_grid_cat, cv=3, scoring='r2')
grid_search_cat.fit(X_train_all, y_train_all)
best_cat = grid_search_cat.best_estimator_

# Model 6: MLPRegressor
# lbfgs (솔버) - 소규모 데이터셋일 경우 사용
mlp = MLPRegressor(random_state=42, max_iter=1500, early_stopping=True, tol=1e-3)
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,)],
    'alpha': [0.0001, 0.001]
}
grid_search_mlp = GridSearchCV(mlp, param_grid_mlp, cv=3, scoring='r2')
grid_search_mlp.fit(X_train_all, y_train_all)
best_mlp = grid_search_mlp.best_estimator_

# 개별 모델 CV R² 계산
cv_ridge = cross_val_score(best_ridge, X_train_all, y_train_all, cv=3, scoring='r2').mean()
cv_rf = cross_val_score(best_rf, X_train_all, y_train_all, cv=3, scoring='r2').mean()
cv_xgb = cross_val_score(best_xgb, X_train_all, y_train_all, cv=3, scoring='r2').mean()
cv_lgb = cross_val_score(best_lgb, X_train_all, y_train_all, cv=3, scoring='r2').mean()
cv_cat = cross_val_score(best_cat, X_train_all, y_train_all, cv=3, scoring='r2').mean()
cv_mlp = cross_val_score(best_mlp, X_train_all, y_train_all, cv=3, scoring='r2').mean()

print("개별 모델 CV R²:")
print("Ridge:", cv_ridge)
print("RandomForest:", cv_rf)
print("XGBoost:", cv_xgb)
print("LightGBM:", cv_lgb)
print("CatBoost:", cv_cat)
print("MLPRegressor:", cv_mlp)

# 앙상블: StackingRegressor (베이스: 모든 모델, 메타: Ridge)
estimators = [
    ('ridge', best_ridge),
    ('rf', best_rf),
    ('xgb', best_xgb),
    ('lgb', best_lgb),
    ('cat', best_cat),
    ('mlp', best_mlp)
]
stacking_reg = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(),
    cv=3,
    n_jobs=-1
)
stacking_reg.fit(X_train_all, y_train_all)
cv_stacking = cross_val_score(stacking_reg, X_train_all, y_train_all, cv=3, scoring='r2').mean()
print("Stacking 앙상블 CV R²:", cv_stacking)

#####################################
# 7. 평가 지표 계산 (테스트 데이터)
#####################################
def evaluate_model(model, X, y):
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    mae = mean_absolute_error(y, y_pred)
    return r2, rmse, mae

models = {
    "Ridge": best_ridge,
    "RandomForest": best_rf,
    "XGBoost": best_xgb,
    "LightGBM": best_lgb,
    "CatBoost": best_cat,
    "MLPRegressor": best_mlp,
    "Stacking": stacking_reg
}

for name, model in models.items():
    r2_val, rmse_val, mae_val = evaluate_model(model, X_test_all, y_test_all)
    print(f"\n{name} 모델 평가:")
    print(f"R²: {r2_val:.4f}")
    print(f"RMSE: {rmse_val:.4f}")
    print(f"MAE: {mae_val:.4f}")

#####################################
# 8. 최종 추천: Stacking 앙상블 모델 사용 (전체 데이터 기반 추천)
#####################################
data_filtered = data_filtered.reset_index(drop=True)
data_filtered['predicted_score'] = stacking_reg.predict(X_all_scaled)
data_filtered['final_score'] = data_filtered['score']  # imputation 후 값 사용

# 추가 보정: composite_score 산출 (리뷰와 유의사항 보정)
for col in ['caution_배달가능', 'caution_예약가능', 'caution_포장가능',
            'caution_배달불가', 'caution_예약불가', 'caution_포장불가']:
    if col not in data_filtered.columns:
        data_filtered[col] = 0

# 'review' 컬럼을 숫자형으로 변환 (문자열이 있을 경우 처리)
data_filtered['review'] = pd.to_numeric(data_filtered['review'], errors='coerce')

def compute_composite_score(row, review_weight=review_weight, caution_weight=caution_weight, convenience_weight=convenience_weight):
    base = row['final_score']

    # 1. 리뷰 보정 로직 수정
    review_val = float(row['review'])
    # 리뷰 수가 적더라도 어느 정도 보정값을 받을 수 있도록 조정
    review_adjust = review_weight * (np.log(review_val + 50) / np.log(1000))

    # 2. 유의사항 보정 로직 강화
    pos = row.get('caution_배달가능', 0) + row.get('caution_예약가능', 0) + row.get('caution_포장가능', 0)
    neg = row.get('caution_배달불가', 0) + row.get('caution_예약불가', 0) + row.get('caution_포장불가', 0)

    # 편의시설 보정: "conv_"로 시작하는 컬럼 중 "conv_편의시설 정보 없음"은 제외
    conv_cols = [col for col in row.index if col.startswith("conv_") and col != "conv_편의시설 정보 없음"]
    if conv_cols and any(row[col] for col in conv_cols):
        conv_mean = np.mean([row[col] for col in conv_cols])
    else:
        conv_mean = 0
    conv_adjust = convenience_weight * conv_mean

    return base + review_adjust + caution_weight * (pos - neg) + conv_adjust

# 먼저 composite_score 계산
data_filtered['composite_score'] = data_filtered.apply(compute_composite_score, axis=1)

# sigmoid 함수 a와 b 값을 설정 (예: a=1.25, b=2.5 또는 원하는 값)
a = 1.25
b = 2.5

# Sigmoid 변환 함수도 수정
def sigmoid_transform(x, a, b):  # b값을 평균에 가깝게 조정
    return 5 * (1 / (1 + np.exp(-a * (x - b))))

# composite_score에 sigmoid 변환 적용
data_filtered['composite_score'] = data_filtered['composite_score'].apply(lambda x: sigmoid_transform(x, a, b))

# 최종 추천: composite_score 기준 내림차순 정렬
recommendations_all = data_filtered.sort_values(by='composite_score', ascending=False)
# 최종 추천 결과 출력 (예: id 컬럼 추가)
print("\n[전체 데이터 기반 추천] 상위 15개:")
print(recommendations_all[['id', 'category_id', 'score', 'predicted_score', 'composite_score']].head(15))

# 상위 15개 결과 추출 (DataFrame recommendations_all이 이미 존재한다고 가정)
top15 = recommendations_all[['id', 'category_id', 'score', 'predicted_score', 'composite_score']].head(15).copy()

# predicted_score와 composite_score를 소수점 3자리까지 반올림
top15['predicted_score'] = top15['predicted_score'].round(3)
top15['composite_score'] = top15['composite_score'].round(3)

# 사용자 정보와 추천 결과를 딕셔너리로 구성
result_dict = {
    "user": user_id,  # 앞에서 입력받은 사용자 ID
    "recommendations": json.loads(top15.to_json(orient='records', force_ascii=False))
}

# 딕셔너리를 JSON 문자열로 변환 (들여쓰기 적용)
result_json = json.dumps(result_dict, ensure_ascii=False, indent=4)

# JSON 문자열 출력
print(result_json)

# JSON 파일로 저장 및 Colab 환경에서 다운로드
with open('/content/recommendations_top15.json', 'w', encoding='utf-8') as f:
    f.write(result_json)

files.download('/content/recommendations_top15.json')


사용자 ID를 입력하세요: toby
선호하는 식당 카테고리 이름(최대 3개, 콤마로 구분)을 입력하세요: 다이닝바, 오마카세, 파스타
개별 모델 CV R²:
Ridge: -0.012135754288020606
RandomForest: -0.13924784785179875
XGBoost: -0.06318262950556501
LightGBM: -0.045066674290735964
CatBoost: -0.03886799752386735
MLPRegressor: -0.4122050285959106
Stacking 앙상블 CV R²: 0.009343805111709994

Ridge 모델 평가:
R²: 0.0290
RMSE: 1.1561
MAE: 0.7759

RandomForest 모델 평가:
R²: 0.0353
RMSE: 1.1524
MAE: 0.7801

XGBoost 모델 평가:
R²: 0.0186
RMSE: 1.1623
MAE: 0.7826

LightGBM 모델 평가:
R²: 0.0602
RMSE: 1.1374
MAE: 0.7754

CatBoost 모델 평가:
R²: 0.0437
RMSE: 1.1473
MAE: 0.7765

MLPRegressor 모델 평가:
R²: -0.0471
RMSE: 1.2006
MAE: 0.8429

Stacking 모델 평가:
R²: 0.0380
RMSE: 1.1508
MAE: 0.7673

[전체 데이터 기반 추천] 상위 15개:
         id  category_id  score  predicted_score  composite_score
167  03-194            3    5.0         3.705841         4.920393
94   03-102            3    5.0         3.833391         4.919741
75    03-82            3    5.0         3.781578         4.918214
150  03-172     

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# 각 카테고리별 상위 5개 식당의 점수 구성요소 확인
for cat_id in [0, 1, 6]:  # 중식, 일식, 한식
    print(f"\n카테고리 {cat_id}의 상위 5개 식당 상세 분석:")
    cat_data = data_filtered[data_filtered['category_id'] == cat_id].sort_values('composite_score', ascending=False).head(5)

    for idx, row in cat_data.iterrows():
        # 각 구성요소 계산
        base = row['final_score']
        review_val = float(row['review'])
        review_adjust = review_weight * np.log(review_val + 1)

        pos = row.get('caution_배달가능', 0) + row.get('caution_예약가능', 0) + row.get('caution_포장가능', 0)
        neg = row.get('caution_배달불가', 0) + row.get('caution_예약불가', 0) + row.get('caution_포장불가', 0)
        caution_adjust = caution_weight * (pos - neg)

        conv_cols = [col for col in row.index if col.startswith("conv_")]
        conv_mean = np.mean([row[col] for col in conv_cols]) if conv_cols else 0
        conv_adjust = convenience_weight * conv_mean

        print(f"\n식당 ID {idx}:")
        print(f"기본 점수: {base:.3f}")
        print(f"리뷰 수: {review_val:.0f}")
        print(f"리뷰 보정: {review_adjust:.3f}")
        print(f"유의사항 보정: {caution_adjust:.3f}")
        print(f"편의시설 보정: {conv_adjust:.3f}")
        print(f"최종 composite_score: {row['composite_score']:.3f}")


카테고리 0의 상위 5개 식당 상세 분석:

카테고리 1의 상위 5개 식당 상세 분석:

식당 ID 207:
기본 점수: 5.000
리뷰 수: 3
리뷰 보정: 0.555
유의사항 보정: 0.300
편의시설 보정: 0.150
최종 composite_score: 4.908

식당 ID 73:
기본 점수: 5.000
리뷰 수: 1
리뷰 보정: 0.277
유의사항 보정: 0.300
편의시설 보정: 0.000
최종 composite_score: 4.889

식당 ID 192:
기본 점수: 5.000
리뷰 수: 14
리뷰 보정: 1.083
유의사항 보정: 0.150
편의시설 보정: 0.075
최종 composite_score: 4.880

식당 ID 151:
기본 점수: 4.800
리뷰 수: 142
리뷰 보정: 1.985
유의사항 보정: 0.300
편의시설 보정: 0.025
최종 composite_score: 4.875

식당 ID 184:
기본 점수: 4.700
리뷰 수: 1
리뷰 보정: 0.277
유의사항 보정: 0.450
편의시설 보정: 0.025
최종 composite_score: 4.871

카테고리 6의 상위 5개 식당 상세 분석:


predicted_score는 모델이 예측한 평점(또는 imputation을 통해 보완된 평점)입니다.
composite_score는 이 예측 평점에 리뷰, 유의사항, 편의시설 등의 보정 값을 반영하여 계산한 최종 추천 평점입니다.

만약 실제 평점이 없는 경우 imputation을 통해 가상 평점(보완된 평점)을 산출하고, 이 가상 평점을 기반으로 모델이 예측한 predicted_score를 계산하게 됩니다.
즉, 데이터에 실제 평점이 없으면, imputation을 통해 보완된 가상 평점이 모델 입력의 기준이 되어 predicted_score가 산출됩니다

결과값 컬럼

- category_id: 식당의 카테고리를 나타내는 숫자형 값입니다.
예를 들어, "한식", "중식", "일식" 등 미리 정의한 매핑에 따라 각 식당이 어느 카테고리에 속하는지를 보여줍니다.

- score: 데이터에 존재하는 실제 평점 값(또는 imputation을 통해 보완된 평점)입니다.
이 값은 식당이 원래 받은 평점을 의미하며, 대부분 상위 추천 식당에서는 4.7~5.0 정도의 높은 평점을 보입니다.

- predicted_score: 모델(예: Stacking 앙상블 등)이 예측한 평점 값입니다.
실제 평점과는 다르게 산출되며, 모델이 학습한 피처들을 기반으로 예측한 결과입니다.
이 값은 모델의 예측 성능을 평가할 때 참고할 수 있으며, 보정 전의 예측치를 나타냅니다.

- composite_score: 최종 추천 점수로, 실제 평점(score)에 리뷰에 대한 보정(예: 로그 변환된 review 값에 기초한 가중치 조정)과 유의사항(caution) 보정을 더한 값입니다.
이 값은 최종적으로 추천 순위를 결정하는 데 사용됩니다.
추가로, 전체 composite_score가 선형 스케일링을 통해 최대 5점으로 조정되었으므로, 이 값이 5점을 넘지 않게 되어 있습니다.
